# Enabling Deep Learning in Gravitational Wave Physics With Inference-as-a-Service
## Alec Gunny

## Deep Learning in Gravitational Wave Astronomy
<img src="images/gw-dl-use-cases.png" height="auto" width="980px" style="top:10px;"></img>

## Inference Time Challenges
<div class="float200 padded">
    <img src="images/hardware-logos.png" width="300px" height="auto" class="right unpadded"/>
    <p class="left">Access to and familiarity with accelerated hardware</p>
</div>

<div>
    <img src="images/framework-logos.png" width="300px" height="auto" class="left padded" />
    <p class="right"> Managing, leveraging, and translating across deep learning software stacks
</div>

<div></div>
<div></div>
<div class="float100">
    <img src="images/distributing-nns.png" width="250px" height="auto" class="right unpadded"/>
    <p class="left">Distributing updated models to dependent users and applications</p>
</div>

## A simple PyTorch example
- Naive inference pipeline for a model trained using PyTorch
- Pipeline itself loads the model and data onto the GPU and executes inference

In [1]:
import time
from concurrent.futures import ThreadPoolExecutor
from queue import Empty, Queue

import numpy as np
import torch

import utils

Instantiate a simple model, "train" it, then save it for inference:

In [2]:
class MLP(torch.nn.Module):
    def __init__(self, input_size, hidden_sizes):
        super().__init__()

        self.layers = torch.nn.ModuleList()
        for size in hidden_sizes:
            self.layers.append(torch.nn.Linear(input_size, size))
            self.layers.append(torch.nn.ReLU())
            input_size = size

        self.layers.append(torch.nn.Linear(input_size, 1))
        self.layers.append(torch.nn.Sigmoid())

    def forward(self, x: torch.tensor) -> torch.tensor:
        for layer in self.layers:
            x = layer(x)
        return x

INPUT_SIZE, HIDDEN_SIZES = 64, [256, 128, 64]
model = MLP(INPUT_SIZE, HIDDEN_SIZES).cuda(device=0)

# do_some_training(model) -> export the trained weights
torch.save(model.state_dict(), "model.pt")

At inference time, load in the model weights and use it to map inputs to outputs

In [3]:
inference_model = MLP(INPUT_SIZE, HIDDEN_SIZES).cuda(0)
inference_model.load_state_dict(torch.load("model.pt"))

# create an array on the CPU
x = np.random.randn(INPUT_SIZE).astype("float32")
with torch.no_grad():
    # move it on to the GPU
    x = torch.from_numpy(x).cuda(0)

    # map it to outputs on the GPU
    y = inference_model(x)

# move the outputs back on to the CPU
y.cpu().numpy()

array([0.53046507], dtype=float32)

- Generally interested in using the model for many thousands or millions of inferences
- Start with the simplest case: a dataset that can fit into memory at once

In [4]:
dataset = np.random.randn(5 * 10**5, 64).astype("float32")

@torch.no_grad()
def do_some_inference(model, dataset, batch_size=8, device_index=0):
    # move the data to the GPU in bulk
    gpu_dataset = torch.from_numpy(dataset).cuda(device_index)

    # iterate through it in batches and yield predictions
    dataset = torch.utils.data.TensorDataset(gpu_dataset)
    for [x] in torch.utils.data.DataLoader(dataset, batch_size=batch_size):
        y = model(x)
        yield y.cpu().numpy()

# run through the dataset and get a rough time estimate
%time outputs = [y for y in do_some_inference(inference_model, dataset)]

CPU times: user 17.3 s, sys: 146 ms, total: 17.5 s
Wall time: 17.5 s


Ok, but how well are we utilizing the GPU?

In [5]:
with utils.GpuUtilProgress(gpu_ids=0) as progbar:
    task_id = progbar.add_task("[cyan]Inference", total=len(dataset))

    outputs = []
    for y in do_some_inference(model, dataset):
        outputs.append(y)
        progbar.update(task_id, advance=len(y))

output = np.concatenate(outputs, axis=0)

Output()

- Not very well! GPUs are expensive, can we improve things via parallel execution (assuming we can't change the batch size)?
- First attempt: Naive (and sloppy) implementation using threading

In [6]:
q = Queue()
def task(dataset_chunk):
    model = MLP(INPUT_SIZE, HIDDEN_SIZES).cuda(0)
    model.load_state_dict(torch.load("model.pt"))

    for y in do_some_inference(model, dataset_chunk):
        q.put(y)

with utils.GpuUtilProgress(0) as progbar, ThreadPoolExecutor(4) as pool:
    task_id = progbar.add_task("Inference with 4 jobs", total=len(dataset))
    [pool.submit(task, x) for x in np.array_split(dataset, 4)]

    outputs = []
    while len(outputs) < len(dataset):
        y = q.get()
        progbar.update(task_id, advance=len(y))
        outputs.extend(y)

Output()

Not much help. What next?

After spending a few hours perusing the PyTorch documentation, we come up with the following basic functional implementation:

In [7]:
def parallel_inference(X, num_gpus, jobs_per_gpu, progbar):
    num_jobs = num_gpus * jobs_per_gpu
    task_id = progbar.add_task(
        f"[cyan]{num_gpus} GPUs/{num_jobs} jobs",
        total=len(X),
        start=False
    )

    # we need special queue and value objects
    # specific to process spawning
    smp = torch.multiprocessing.get_context("spawn")
    q = smp.Queue()
    sync = smp.Value("d", 0.0)

    # pass a bunch of arguments into each
    # process that we need to spawn
    # note that we have to pass copies of some
    # of our local functions that live in `utils`
    # since we can't unpickle elsewhere functions
    # which are defined in __main__
    args = (
        X,  # the full dataset
        utils.MLP,  # the module class to use for inference
        [INPUT_SIZE, HIDDEN_SIZES],  # arguments to initialize the module
        utils.do_some_inference,  # the inference funcntion to use
        q,  # the queue to put the results in
        sync,  # a task synchronizer
        jobs_per_gpu,
        num_gpus
    )

    # spawn parallel jobs across all GPUs.
    # We have to host the `parallel_inference_task` function
    # in a separate module for the same pickling pickle
    # described above
    procs = torch.multiprocessing.spawn(
        utils.parallel_inference_task,
        args=args,
        nprocs=num_jobs,
        join=False
    )

    # wait to synchronize until all models load
    # so that we can compare throughput better
    while sync.value < num_jobs:
        time.sleep(0.01)

    # increment the synchronizer one more
    # time to kick off the jobs
    sync.value += 1
    progbar.start_task(task_id)

    # collect all the (unordered) inputs
    outputs = []
    while True:
        try:
            # try to get the next result in
            # in the queue and increment everything
            y = q.get_nowait()
            progbar.update(task_id, advance=len(y))
            outputs.append(y)
        except Empty:
            # if there's nothing in the queue and
            # all the jobs are dead, we're done
            if procs.join(0.01):
                break

    # concatenate the outputs and return
    return np.concatenate(outputs, axis=0)

Run this at a few different scales to see what we can achieve:

In [8]:
with utils.GpuUtilProgress([0, 1]) as progbar:
    y = parallel_inference(dataset, num_gpus=1, jobs_per_gpu=2, progbar=progbar)
    y = parallel_inference(dataset, num_gpus=1, jobs_per_gpu=4, progbar=progbar)
    y = parallel_inference(dataset, num_gpus=2, jobs_per_gpu=4, progbar=progbar)

Output()

It manages to increase GPU utilization pretty well! But:
- Framework specific
    - No help if we want to extend to other frameworks
    - Torch is pretty unique in having this functionality at all
- The code is complicated and required a lot of non-physics expertise to build
    - Non-trivial to reconstruct for new applications
- Extremely contrived example, breaks down in most real use cases
    - Explore a few cases to show how

## Throughput too low
#### _The constraints of our use case demand that we further reduce processing time by an order of magnitude_
- Not obvious how to simply extend this code to multi-node
- Scaling not dynamic
    - Have to pick a level of parallelism and hope the resources are available to use it

## Throughput too high
####  _Data generation process is slow, needs to be parallelized to saturate GPU throughput_

In [5]:
class ThrottledDataset(torch.utils.data.IterableDataset):
    def __init__(self, x, device_index=0):
        self.x = torch.from_numpy(x).cuda(device_index)

    def __iter__(self):
        self.it = iter(self.x)
        return self

    def __next__(self):
        x = next(self.it)
        time.sleep(0.001)
        return x


@torch.no_grad()
def do_some_throttled_inference(model, dataset, batch_size=8, device_index=0):
    gpu_dataset = ThrottledDataset(dataset, device_index)
    for x in torch.utils.data.DataLoader(gpu_dataset, batch_size=batch_size):
        y = model(x)
        yield y.cpu().numpy()

## Throughput too high
Reimplement parallel inference with our new inference function

In [6]:
def throttled_parallel_inference(X, num_gpus, jobs_per_gpu, progbar):
    num_jobs = num_gpus * jobs_per_gpu
    task_id = progbar.add_task(
        f"[cyan]{num_gpus} GPUs/{num_jobs} jobs",
        total=len(X),
        start=False
    )

    # we need special queue and value objects
    # specific to process spawning
    smp = torch.multiprocessing.get_context("spawn")
    q = smp.Queue()
    sync = smp.Value("d", 0.0)

    # pass a bunch of arguments into each
    # process that we need to spawn
    # note that we have to pass copies of some
    # of our local functions that live in `utils`
    # since we can't pickle functions defined
    # in __main__
    args = (
        X,  # the full dataset
        utils.MLP,  # the module class to use for inference
        [INPUT_SIZE, HIDDEN_SIZES],  # arguments to initialize the module
        utils.do_some_throttled_inference,  # the inference funcntion to use
        q,  # the queue to put the results in
        sync,  # a task synchronizer
        jobs_per_gpu,
        num_gpus
    )

    # spawn a bunch of parallel jobs across all GPUs
    # have to host the `parallel_inference_task` in
    # a separate module for weird multiprocessing reasons
    procs = torch.multiprocessing.spawn(
        utils.parallel_inference_task,
        args=args,
        nprocs=num_jobs,
        join=False
    )

    # wait to synchronize until all models load
    # so that we can compare throughput better
    while sync.value < num_jobs:
        time.sleep(0.01)

    # increment the starter to kick off the jobs
    sync.value += 1
    progbar.start_task(task_id)

    # collect all the (unordered) inputs
    outputs = []
    while True:
        try:
            y = q.get_nowait()
            progbar.update(task_id, advance=len(y))
            outputs.append(y)
        except Empty:
            # if there's nothing in the queue and
            # all the jobs are dead, we're done
            if procs.join(0.01):
                break

    # concatenate the outputs and return
    return np.concatenate(outputs, axis=0)

## Throughput too high

In [7]:
x = dataset[:2 * 10**4]  # this will take longer, so just do a few for demo purposes
with utils.GpuUtilProgress([0, 1]) as progbar:
    y = throttled_parallel_inference(x, 1, 1, progbar=progbar)
    y = throttled_parallel_inference(x, 1, 2, progbar=progbar)
    y = throttled_parallel_inference(x, 1, 4, progbar=progbar)

Output()

## Model ensembling
#### _Connecting multiple models in a single pipeline_
![Model sharing](images/model_sharing.png "Model sharing")
![Model ensemble](images/model_ensemble.png "Model ensemble")

## Model ensembling
Naive implemenation

In [12]:
@torch.no_grad()
def do_some_multi_model_inference(models, dataset, batch_size=8, device_index=0):
    gpu_dataset = torch.from_numpy(dataset).cuda(device_index)
    dataset = torch.utils.data.TensorDataset(gpu_dataset)

    for [x] in torch.utils.data.DataLoader(dataset, batch_size=batch_size):
        for model in models:
            x = model(x)
        yield x.cpu().numpy()

noise_remover = utils.NoiseRemovalModel(INPUT_SIZE, [32, 16]).cuda(0)
models = [noise_remover, inference_model]

with utils.GpuUtilProgress(0) as progbar:
    task_id = progbar.add_task("[cyan]Inference", total=len(dataset))
    for y in do_some_multi_model_inference(models, dataset):
        progbar.update(task_id, advance=len(y))

Output()

## Model Ensembling
Extension to parallel execution non-trivial:
- Each model should execute asynchronously
- Usually will need _different_ levels of parallelism to alleviate bottlenecks

![Bottleneck](images/bottleneck.png "Bottleneck")
![Alleviated bottleneck](images/bottleneck-alleviate.png "Alleviated bottleneck")

- Pass tensors between GPUs to best utilize hardware
- Almost impossible if the other model utilizes a different framework

## Distribution
What if you want someone with no background in any of this to use your application? Or more likely, to plug it into their application?

## Inference-as-a-Service